embedder code...

import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
from pytorch_wavelets import DWTForward, DWTInverse
from google.colab import files  # For image upload in Colab

# -------------------------------
# Device
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------
# Embedder Model
# -------------------------------
class Embedder(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1,16,3,2,1),
            nn.ReLU(),
            nn.Conv2d(16,32,3,2,1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return 0.002 + self.net(x) * 0.018  # alpha range

# -------------------------------
# DWT layers
# -------------------------------
dwt = DWTForward(J=1, wave='haar').to(device)
idwt = DWTInverse(wave='haar').to(device)

# -------------------------------
# Utilities
# -------------------------------
def load_gray(path, size=512):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (size, size))
    img = torch.tensor(img/255.0, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    return img.to(device)

def embed_frequency(host, wm, alpha):
    # DWT decomposition
    Yl, Yh_list = dwt(host)
    Yh = Yh_list[0]  # Yh: [B, C, 3, H, W]

    # Extract subbands correctly
    LH = Yh[:, :, 0, :, :]
    HL = Yh[:, :, 1, :, :]
    HH = Yh[:, :, 2, :, :]

    # Resize watermark to match LH/HL
    wm_resized = F.interpolate(wm, size=LH.shape[-2:], mode='bilinear')

    # Embed watermark
    LH_emb = LH + alpha * wm_resized
    HL_emb = HL + alpha * wm_resized

    # Reconstruct Yh with embedded subbands
    Yh_emb = torch.stack([LH_emb, HL_emb, HH], dim=2)  # shape: [B, C, 3, H, W]

    # Inverse DWT
    wm_img = idwt((Yl, [Yh_emb]))
    return wm_img

# -------------------------------
# Load model
# -------------------------------
embedder = Embedder().to(device)
embedder.load_state_dict(torch.load("embedder_model.pth", map_location=device))
embedder.eval()

# -------------------------------
# Upload images
# -------------------------------
print("Upload your host image:")
uploaded = files.upload()
host_path = list(uploaded.keys())[0]

print("Upload your watermark image:")
uploaded = files.upload()
wm_path = list(uploaded.keys())[0]

host = load_gray(host_path)
wm = load_gray(wm_path, size=host.shape[-1])

# ---- Embed ----
with torch.no_grad():
    alpha = embedder(host)
    wm_img = embed_frequency(host, wm, alpha)

# ---- Save watermarked image ----
wm_img_np = wm_img.squeeze().cpu().numpy()
cv2.imwrite("watermarked_image.png", (wm_img_np*255).astype('uint8'))
print("Watermarked image saved as watermarked_image.png")


In [ ]:
extractor...

In [ ]:
import torch
import torch.nn as nn
import cv2
import torch.nn.functional as F
from pytorch_wavelets import DWTForward
from google.colab import files  # For image upload in Colab

# -------------------------------
# Device
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------
# Utilities
# -------------------------------
def load_gray(path, size=512):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (size, size))
    img = torch.tensor(img/255.0, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    return img.to(device)

# -------------------------------
# DWT forward
# -------------------------------
dwt = DWTForward(J=1, wave='haar').to(device)

# -------------------------------
# Extract watermark
# -------------------------------
def extract_frequency(wm_img, host, alpha):
    Yl_host, Yh_host_list = dwt(host)
    Yl_wm, Yh_wm_list = dwt(wm_img)

    Yh_host = Yh_host_list[0]
    Yh_wm = Yh_wm_list[0]

    LH_host = Yh_host[:, :, 0, :, :]
    HL_host = Yh_host[:, :, 1, :, :]

    LH_wm = Yh_wm[:, :, 0, :, :]
    HL_wm = Yh_wm[:, :, 1, :, :]

    # Extract watermark from LH and HL
    wm_extracted = ( (LH_wm - LH_host) + (HL_wm - HL_host) ) / (2*alpha)
    return wm_extracted

# -------------------------------
# Upload images
# -------------------------------
print("Upload the watermarked image:")
uploaded = files.upload()
wm_img_path = list(uploaded.keys())[0]

print("Upload the original host image:")
uploaded = files.upload()
host_path = list(uploaded.keys())[0]

# -------------------------------
# Load images
# -------------------------------
wm_img = load_gray(wm_img_path)
host = load_gray(host_path)

# -------------------------------
# Set alpha (must match the one used in embedder)
# -------------------------------
alpha_value = 0.01  # You can adjust or calculate from embedder output

# -------------------------------
# Extract watermark
# -------------------------------
with torch.no_grad():
    wm_extracted = extract_frequency(wm_img, host, alpha_value)

# ---- Save extracted watermark ----
wm_extracted_np = wm_extracted.squeeze().cpu().numpy()
cv2.imwrite("extracted_watermark.png", (wm_extracted_np*255).astype('uint8'))
print("Extracted watermark saved as extracted_watermark.png")
